# Make a set of Amazon Personalize campaigns
Now that the data has been generated, the datasets have been created, and the
data has been imported, now it is time to create Personalize solutions, solution
versions, and campaigns.

In [24]:
import json
import boto3
import time

cars_filename         = 'car_items.csv'
users_filename        = 'users.csv'
interactions_filename = 'interactions.csv'

region      = '<region>'
account_num = '<your-account>'
dataset_group_name = 'car-dg10'

dataset_group_arn = 'arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, account_num, dataset_group_name)

MAX_WAIT_TIME = 60*60 # 1 hour

role_arn = 'arn:aws:iam::355151823911:role/PersonalizeS3Role'

In [25]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Function to create a new campaign for a specific recipe
This function creates a new campaign in our dataset group based on a specific
Personalize recipe.

In [34]:
def make_campaign(recipe_arn, name, dataset_group_arn):
    print('Entered make_campaign for {}'.format(name))
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn
    )
    solution_arn = create_solution_response['solutionArn']
    print('created solution: {}'.format(solution_arn))

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )
    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('Waiting for solution version to become active...')
    latest_time = time.time() + MAX_WAIT_TIME
    while time.time() < latest_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response['solutionVersion']['status']
        print('SolutionVersion: {} - {}'.format(name, status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            if status == 'CREATE FAILED':
                print('*** Solution Version Create failed ***')
            break

        time.sleep(60)
    get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn
    )

    print(json.dumps(get_solution_metrics_response, indent=2))    

    create_campaign_response = personalize.create_campaign(
        name = name,
        solutionVersionArn = solution_version_arn,
        minProvisionedTPS = 1
    )
    campaign_arn = create_campaign_response['campaignArn']
    print('Waiting for campaign to become active...')

    latest_time = time.time() + MAX_WAIT_TIME
    while time.time() < latest_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = describe_campaign_response['campaign']['status']
        print('Campaign: {} - {}'.format(name, status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            break

        time.sleep(60)
        
    print('Exiting make_campaign for {}'.format(name))
    return(solution_arn, solution_version_arn, campaign_arn)

In [27]:
#make_campaign('arn:aws:personalize:::recipe/aws-hrnn', 'car-hrnn', dataset_group_arn)

In [28]:
#make_campaign('arn:aws:personalize:::recipe/aws-sims', 'car-sims', dataset_group_arn)

In [29]:
#make_campaign('arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking', dataset_group_arn)

In [30]:
#make_campaign('arn:aws:personalize:::recipe/aws-hrnn-metadata', 'car-hrnn-metadata', dataset_group_arn)

## Create multiple campaigns in parallel
Creating solutions and campaigns can be time-consuming depending on the size and
complexity of the datasets and the chosen recipe. Here we create a set of campaigns
in parallel. In practice, a given application may only need one or two campaigns.
However, in development and testing, you may want to try out a full suite of recipes
to find out how well they serve your specific requirements and data.

In [31]:
from multiprocessing import Process

def make_campaigns_in_parallel(campaigns, dg_arn):
    jobs = []
    for i in campaigns:
        p = Process(target = make_campaign, args=(i[0], i[1], dg_arn))
        jobs.append(p)
        
    for p in jobs:
        p.start()
        
    for p in jobs:
        p.join()

In [32]:
campaigns = [['arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking'],
             ['arn:aws:personalize:::recipe/aws-sims',                 'car-sims'],
             ['arn:aws:personalize:::recipe/aws-popularity-count',     'car-popularity-count'],
             ['arn:aws:personalize:::recipe/aws-hrnn-metadata',        'car-hrnn-metadata'],
             ['arn:aws:personalize:::recipe/aws-hrnn',                 'car-hrnn']
            ]
print(dataset_group_arn)
make_campaigns_in_parallel(campaigns, dataset_group_arn)

arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg10
Entered make_campaign for car-personalized-ranking
Entered make_campaign for car-sims
Entered make_campaign for car-popularity-count
Entered make_campaign for car-hrnn-metadata
Entered make_campaign for car-hrnn
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-popularity-count
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn-metadata
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-sims
Waiting for solution version to become active...
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn
Waiting for solution version to become active...
Waiting for solution version to become active...
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE PENDING
Waiting for solution version to become active..

SolutionVersion: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: ACTIVE
{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking/ac0c6f2a",
  "metrics": {
    "coverage": 0.0008,
    "mean_reciprocal_rank_at_25": 0.0014,
    "normalized_discounted_cumulative_gain_at_10": 0.0029,
    "normalized_discounted_cumulative_gain_at_25": 0.0031,
    "n